In [ ]:
import os
import cv2
import json
import numpy as np
import pandas
import skimage
import shapefile
import matplotlib.pyplot as plt

import modules

### load data

In [ ]:
kenya_osm, kenya_sf = modules.data.load_shapefile("kenya")
kenya_geo = modules.data.load_geodata("kenya")
kenya_dat = pandas.DataFrame.merge(kenya_geo, kenya_osm, on="index")

kenya_filenames = modules.data.util.load_image_filenames("kenya", D=128)

In [ ]:
peru_osm, peru_sf = modules.data.load_shapefile("peru")
peru_geo = modules.data.load_geodata("peru")
peru_dat = pandas.DataFrame.merge(peru_geo, peru_osm, on="index")
# peru_filenames = modules.data.util.load_image_filenames("peru", D=128)